### Load all of the packages

### Download the HTML of a Certain Page and Name

In [1]:
from bs4 import BeautifulSoup
import requests
import sys
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
def get_file(first_name, last_name, page):
    # define search url based on user input
    url = (
        f"https://www.findagrave.com/memorial/search?firstname={first_name}&"
        f"middlename=&lastname={last_name}&birthyear=&birthyearfilter=&"
        "deathyear=&deathyearfilter=&location=&locationId=&memorialid=&mcid=&"
        f"linkedToName=&datefilter=&orderby=r&plot=&page={page}#sr-1075")

    page_HTML = requests.get(url) # HTML from URL

    # successful url request code is 200
    if page_HTML.status_code != 200:
        sys.exit("Connection Failed.") # stop execution and return error

    return page_HTML

### Go Through All Entries for a Certain Name and Return a Dataframe

In [110]:
def get_info(first_name="George", last_name="Washington"):
    # Define the empty dataframe
    data_table = pd.DataFrame(columns = ["Names", "Birth Year", "Death Year", "Location of Grave"])

    # get HTML of first page
    page = get_file(first_name, last_name, "1")
    # parse HTML into Beautiful Soup format
    parsed = BeautifulSoup(page.content, "html.parser")

    # find the div containing the maximum number of pages and specify the max
    # element in the BeautifulSoup data structure as if it was a dictionary with
    # a key value "max"
    max_page = (parsed.find(id = "gotoPage"))["max"]

    # collect data from all pages
    for page_number in range(int(max_page)):

        # get HTML of specified page number
        page = get_file(first_name, last_name, str(page_number + 1))

        # parse HTML into Beautiful Soup format
        parsed = BeautifulSoup(page.content, "html.parser")

        # each memorial-item is a tag for a grave entry in the database
        # HTML note: tags act like containers that contain attributes and text
        memorial_list = parsed.find_all(class_ = "memorial-item")

        # Iterate through each memorial and only collect useful data
        for memorial_item in memorial_list:
            # double check that first name is George and last name is Washington
            # A blank space after first name eliminates the possibility of names like "Georgette"
            if memorial_item.find("i") is not None and memorial_item.find("i").string is not None and first_name+' ' in memorial_item.find("i").string and ''+ last_name in memorial_item.find("i").string:
                # if it passes all these tests, take the the data from this memorial
                name = memorial_item.find("i").string
               
                # grave address
                if memorial_item.find("p", attrs = {'class':'addr-cemet'}) is not None:
                    # find the text in <p> with class = addr-cemet
                    grave_address = memorial_item.find("p", attrs = {'class':'addr-cemet'}).string
                else: grave_address = "NA"

                # dates of birth and death
                if memorial_item.find(class_ = "birthDeathDates") is not None and "unknown" not in memorial_item.find(class_ = "birthDeathDates").string:
                    # dates of birth/death are stored in class "birthDeathDates"
                    date = memorial_item.find(class_ = "birthDeathDates").string
                
                    # split birth and death dates
                    birth_death_raw = date.split("–")

                    # clean up date formatting
                    for index, date in enumerate(birth_death_raw):
                        # get rid of letters
                        date = "".join(filter(str.isdigit, date))
                        # as long as the string is greater than 3 charaters (a
                        # month abbreviation) take the last 4 characters (the
                        # year)
                        date = date.strip()
                        if len(date) > 3:
                            date = date[-4:]
                        else:
                            date = "NA"
                        birth_death[index] = date

                    birth_year = birth_death[0]
                    death_year = birth_death[1]

                else:
                    birth_year = "NA"
                    death_year = "NA"

                # record scraped data in data frame
                data_table = data_table.append({"Names": name.strip(), "Birth Year": birth_year, "Death Year": death_year, "Location of Grave": grave_address}, ignore_index= True)
    
    
    #! get rid of any NA values
    data_table = data_table.dropna()

    return data_table

In [111]:
data = get_info()

In [5]:
# add column for years lived
# data =
# data["Birth_Year"] = pd.to_numeric(data["Birth_Year"])


# data["Death_Year"] - data["Birth_Year"]
birth_stats = birth_stats.astype(int)

Create new data frame just for analyzing birth statistics

In [34]:
#count the number of times named individual was born for each year
birth_stats = data["Birth Year"].value_counts()

#convert the count series into a dataframe
birth_stats = birth_stats.to_frame()

#for some reason count makes the objects it is counting an index so we need to make them a column
birth_stats.reset_index(inplace=True)

# rename columns for better clarity
birth_stats = birth_stats.rename(columns={"index": "Year", "Birth Year": "Birth Count"})

# filter out and delete entries with months instead of years
birth_stats = birth_stats[~birth_stats.Year.str.contains("A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z")]

# convert all years and birth counts to integers
birth_stats = birth_stats.astype(int)

In [107]:
txt = "9 Oct 1910 – 21 Oct 1951"

txt2 = "Jan – Mar 1905"

birth_death = txt.split("–")

# as long as the string is greater than 3 charaters (a month abbreviation) take
# the last 4 characters (the year)
for index, date in enumerate(birth_death):
    date = "".join(filter(str.isdigit, date))
    date = date.strip()
    print(date)
    
    if len(date) > 3:
        date = date[-4:]
    else:
        date = "NA"
    birth_death[index] = date

# convert years to integers for use later
birth_year = birth_death[0]
death_year = birth_death[1]

print(birth_death)




91910
211951
['1910', '1951']
